In [1]:
import pandas as pd
from transformers import AutoTokenizer, RobertaModel
import re
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
import random
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.spatial.distance import cityblock

train = pd.read_excel("train.xlsx")
test = pd.read_excel("test.xlsx")

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
def remove_special_chars(text):
    text = re.sub(r'[^\w\s#@]', '', text)
    return text

In [4]:
def pickSame(user):
    i = random.randint(0,len(train)-1)
    tweetUser = train.iloc[i]['user']
    cpt =0
    while user != tweetUser or cpt > 10000:
        cpt += 1
        i = random.randint(0,len(train)-1)
        tweetUser = train.iloc[i]['user']
    return train.iloc[i]['text']
def pickDiff(user):
    i = random.randint(0,len(train)-1)
    tweetUser = train.iloc[i]['user']
    cpt =0
    while user == tweetUser or cpt > 10000:
        cpt += 1
        i = random.randint(0,len(train)-1)
        tweetUser = train.iloc[i]['user']
    return train.iloc[i]['text']
def pickRandom(method):
    if method == 'train':
        i = random.randint(0,len(train)-1)
        j = random.randint(0,len(train)-1)
        tweet1 = train.iloc[i]['text']
        tweet2 = train.iloc[j]['text']
    elif method == 'test' :
        i = random.randint(0,len(test)-1)
        j = random.randint(0,len(test)-1)
        tweet1 = test.iloc[i]['text']
        tweet2 = test.iloc[j]['text']
    return tweet1, tweet2, i ,j

In [5]:
trainData = pd.DataFrame(columns=['tweet1', 'tweet2', 'sameUser'])
for k in range(int(len(train)/4)):
    
    tweet1, tweet2 ,i ,j= pickRandom('train')
    if train.iloc[i]['user'] == train.iloc[j]['user']:
        same_user = 1
        tweet3 = pickDiff(train.iloc[i]['user'])
        trainData = pd.concat([trainData, pd.DataFrame({'tweet1': [tweet1], 'tweet2': [tweet3], 'sameUser': [0]})])
    else:
        same_user = 0
        tweet3 = pickSame(train.iloc[i]['user'])
        trainData = pd.concat([trainData, pd.DataFrame({'tweet1': [tweet1], 'tweet2': [tweet3], 'sameUser': [1]})])
    trainData = pd.concat([trainData, pd.DataFrame({'tweet1': [tweet1], 'tweet2': [tweet2], 'sameUser': [same_user]})])

testData = pd.DataFrame(columns=['tweet1', 'tweet2', 'sameUser'])
for k in range(int(len(test)/2)):
    tweet1, tweet2 ,i ,j= pickRandom('test')
    if test.iloc[i]['user'] == test.iloc[j]['user']:
        same_user = 1
        tweet3 = pickDiff(train.iloc[i]['user'])
        testData = pd.concat([testData, pd.DataFrame({'tweet1': [tweet1], 'tweet2': [tweet3], 'sameUser': [0]})])
    else:
        same_user = 0
        tweet3 = pickSame(train.iloc[i]['user'])
        testData = pd.concat([testData, pd.DataFrame({'tweet1': [tweet1], 'tweet2': [tweet3], 'sameUser': [1]})])
    testData = pd.concat([testData, pd.DataFrame({'tweet1': [tweet1], 'tweet2': [tweet2], 'sameUser': [same_user]})])


# nradohom gir horof o nkhaliw hachtag o arobaze
testData['tweet1'] = testData['tweet1'].apply(remove_special_chars)
#nradohom lower case 
testData['tweet1'] = testData['tweet1'].str.lower()
# nradohom gir horof o nkhaliw hachtag o arobaze
testData['tweet2'] = testData['tweet2'].apply(remove_special_chars)
#nradohom lower case 
testData['tweet2'] = testData['tweet2'].str.lower()


# nradohom gir horof o nkhaliw hachtag o arobaze
trainData['tweet1'] = trainData['tweet1'].apply(remove_special_chars)
#nradohom lower case 
trainData['tweet1'] = trainData['tweet1'].str.lower()
# nradohom gir horof o nkhaliw hachtag o arobaze
trainData['tweet2'] = trainData['tweet2'].apply(remove_special_chars)
#nradohom lower case 
trainData['tweet2'] = trainData['tweet2'].str.lower()   

In [6]:
trainData

,tweet1,tweet2,sameUser
0,foreign policy leaders explain why president o...,go behind the scenes on the campaign trail on ...,1
0,foreign policy leaders explain why president o...,2 pockets,0
0,major story that the dems are making up phony ...,economic confidence is soaring as we unleash t...,1
0,major story that the dems are making up phony ...,#happyfathersday mom i love u and thank u,0
0,proud of my girl @tamarbraxtonher her album lo...,#18daystillartpop look at me now i feel on top...,1
...,...,...,...
0,join me on @greta from indianapolis indiana at...,heres my view right now rt @portiaderossi you ...,0
0,@bieberarmy #fanmade lyric vid nice thanks for...,going to stay in the top 3 on billboard this w...,1
0,@bieberarmy #fanmade lyric vid nice thanks for...,sundays top 5 plays #nbaplayoffspictwittercomk...,0
0,oy aussies you made my summer song the #1 song...,get a whiff of this my new fragrance is coming...,1


In [33]:
def Transformer(text1, text2,model,tokenizer):


    encoded_input1 = tokenizer(text1, return_tensors="pt", padding=True, truncation=True)
    encoded_input2 = tokenizer(text2, return_tensors="pt", padding=True, truncation=True)


    with torch.no_grad():
        outputs1 = model(**encoded_input1)
        outputs2 = model(**encoded_input2)

    sentence_representation1 = outputs1.last_hidden_state.mean(dim=1)
    sentence_representation2 = outputs2.last_hidden_state.mean(dim=1)
    MDist = cityblock(sentence_representation1, sentence_representation2)
    return MDist

In [35]:
trainData[trainData['sameUser'] == 1]

,tweet1,tweet2,sameUser
0,foreign policy leaders explain why president o...,go behind the scenes on the campaign trail on ...,1
0,major story that the dems are making up phony ...,economic confidence is soaring as we unleash t...,1
0,proud of my girl @tamarbraxtonher her album lo...,#18daystillartpop look at me now i feel on top...,1
0,tesla dual motor cars are also allwheel drive ...,right question to ask dyson or others what is ...,1
0,when dani learned what genderqueer meant it wa...,@asmnzf we see what you did there and we like it,1
...,...,...,...
0,go guys rt @vivaronaldo lets vote #vivaronaldo...,ontem conseguirmos uma importante vitória para...,1
0,great show again in sydney love australia than...,httpyoutubelujn3rpkcky,1
0,join me on @greta from indianapolis indiana at...,stock market hit another alltime high yesterda...,1
0,@bieberarmy #fanmade lyric vid nice thanks for...,going to stay in the top 3 on billboard this w...,1


In [62]:
dense_layer = nn.Sequential(
    nn.Linear(1, 50),
    nn.Sigmoid(),
    nn.Linear(50, 1),
    nn.Sigmoid()
)
loss_function = nn.BCELoss()
optimizer = optim.Adam(dense_layer.parameters(), lr=0.007)
accuracyCpt = 0
cpt3 =0
cpt2 = 0
model = RobertaModel.from_pretrained("FacebookAI/roberta-base")
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
for step in range(10000):
    randomTweet = random.randint(0,len(trainData)-1)
    x1= trainData.iloc[randomTweet]['tweet1']
    x2= trainData.iloc[randomTweet]['tweet2']
    res = trainData.iloc[randomTweet]['sameUser']
    
    distance = Transformer(x1, x2,model,tokenizer)
    pred = dense_layer(torch.Tensor([distance]))
    loss = loss_function(pred, torch.Tensor([res]))
    if round(pred.item()) == res:
        accuracyCpt +=1
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if(round(pred.item()) == 1):
        cpt2 += 1
    else:
        cpt3 += 1
    # Print loss
    if (step + 1) % 100 == 0:
        print(f'Epoch [{step+1}/{10000}], Loss: {loss.item():.4f}, Accuracy : {accuracyCpt/step}')

Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch [100/10000], Loss: 0.5149, Accuracy : 0.47474747474747475
Epoch [200/10000], Loss: 1.3169, Accuracy : 0.5175879396984925
Epoch [300/10000], Loss: 0.6689, Accuracy : 0.5016722408026756
Epoch [400/10000], Loss: 0.7965, Accuracy : 0.5012531328320802
Epoch [500/10000], Loss: 1.0124, Accuracy : 0.501002004008016
Epoch [600/10000], Loss: 0.8256, Accuracy : 0.5075125208681135
Epoch [700/10000], Loss: 0.6749, Accuracy : 0.5064377682403434
Epoch [800/10000], Loss: 0.7238, Accuracy : 0.5031289111389237
Epoch [900/10000], Loss: 0.5448, Accuracy : 0.5072302558398221
Epoch [1000/10000], Loss: 0.6489, Accuracy : 0.5035035035035035
Epoch [1100/10000], Loss: 1.0792, Accuracy : 0.49954504094631486
Epoch [1200/10000], Loss: 0.2975, Accuracy : 0.5054211843202668
Epoch [1300/10000], Loss: 0.9099, Accuracy : 0.5065434949961509
Epoch [1400/10000], Loss: 0.4052, Accuracy : 0.5110793423874196
Epoch [1500/10000], Loss: 1.1399, Accuracy : 0.5063375583722481
Epoch [1600/10000], Loss: 1.0331, Accuracy : 0.5

In [63]:
from sklearn.metrics import precision_score, recall_score, f1_score

true_labels = []
predicted_labels = []

for idx in range(len(testData)):
    x1 = testData.iloc[idx]['tweet1']
    x2 = testData.iloc[idx]['tweet2']
    true_label = testData.iloc[idx]['sameUser']
    
    distance = Transformer(x1, x2, model, tokenizer)

    pred = dense_layer(torch.Tensor([distance]))
    
    predicted_label = round(pred.item())

    true_labels.append(true_label)
    predicted_labels.append(predicted_label)

precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


Precision: 0.5
Recall: 1.0
F1 Score: 0.6666666666666666
